#📌 Extracão

In [ ]:
import pandas as pd

#Convertendo os Arquivos para análise em um Data Frame
dado = pd.read_json('G:/Meu Drive/Oracle_alura/ETL G8/Challenge_2/Dados/TelecomX_Data.json')
print(dado.head(),'\n\n')


#Normalizando o arquivo
normal = pd.json_normalize(dado.to_dict(orient='records'))
print(normal)

#🔧 Transformação

In [ ]:
#Verificando valores nulos e duplicados
print('NULOS ->',normal.isnull().sum())
print('DUPLICADOS',normal.duplicated().sum())

#Valores unicos em cada coluna
for coluna in normal.columns:
    unico=normal[coluna].unique()
    print(f'\n\nValores unicos da coluna {coluna}->{unico} \n\n')



#Transformando certas colunas para Float
normal['account.Charges.Total']=pd.to_numeric(normal['account.Charges.Total'], errors='coerce')
normal['customer.tenure']=pd.to_numeric(normal['customer.tenure'], errors='coerce')

#Criando uma coluna para gastos diarios
normal=normal.drop('Total.Day', errors='ignore')
normal['Total.Day']=((normal['account.Charges.Total'] / normal['customer.tenure'])/30).round(2)
normal.insert(19,'Total.Day', normal.pop('Total.Day'))

#Removendo os valores vazios da coluna Churn
normal.query('Churn != ""', inplace=True)
print('\n\nValores unicos da coluna Churn', normal['Churn'].unique(),'\n\n')


#📊 Carga e análise

In [ ]:

#Estatisticas descritivas
descrito = normal.describe()

mean = descrito.loc['mean']
median = descrito.loc['50%']
std = descrito.loc['std']
print(f'Mediana\n{median}\n\n media\n{mean}\n\n std\n{std}\n\n')


#Fazendo os gráficos pra perceber os motivos das saida
import plotly.express as px
import matplotlib.pyplot as plt
print(normal.info())


#Gráficos em função do dado Churn
#Tenure
fig, ax = plt.subplots(figsize=(8,5))

churn_yes = normal[normal['Churn'] == 'Yes']['customer.tenure'].astype('Int64')
churn_no  = normal[normal['Churn'] == 'No']['customer.tenure'].astype('Int64')

ax.hist([churn_yes, churn_no], bins=20, label=['Churn: Sim', 'Churn: Não'], stacked=True)
ax.set_title('Histograma de Tenure por Churn')
ax.set_xlabel('Meses de permanência')
ax.set_ylabel('Número de clientes')
ax.legend()
plt.show()

#Outras variaveis não numericas
def churn_grafico(coluna, color):
    fig, ax = plt.subplots(figsize=(8,5))
    churn_yes = normal[normal['Churn'] == 'Yes'][coluna]
    churn_no  = normal[normal['Churn'] == 'No'][coluna]
    
    ax.hist([churn_yes, churn_no], bins=50, label=['Churn: Sim', 'Churn: Não'], stacked=True, color=color)
    ax.set_title(f'Histograma de {coluna} por Churn')
    ax.set_xlabel(coluna)
    ax.set_ylabel('Número de clientes')
    ax.legend()
    plt.show()

#Contagem de evasão variaveis numéricas
churn_grafico('account.Charges.Total', ['red', 'gold'])
churn_grafico('Total.Day', ['#4DF6FA', '#FA8F4D'])


#Contagem de evasão variaveis categóricas
churn_grafico('customer.gender', ['#722BFA','#FAEF2A'])
churn_grafico('account.Contract', ['#FA1E58', '#1EFA3C'])
churn_grafico('account.PaymentMethod', ['#D33FFA', '#B4FA3E'])
churn_grafico('internet.InternetService', ['#FA573A', '#3CFAA9'])



#📄Relatorio Final


### 📘 Introdução
Neste desafio, me foi proposta a tarefa de realizar uma análise da evasão de clientes da empresa **Telecom X**, uma empresa de serviços de Televisão e internet, o projeto se chama **"Churn de Clientes"**. A empresa enfrenta um alto cancelamento de seus serviços por parte dos cliêntes e precisa entender o porquê desta ocorrencia.
A análise busca identificar padrões em dados fornecidos sobre os clientes e correlacioná-los com a variável `Churn`, que indica se o cliente deixou ou não de utilizar os serviços da empresa.

---

### 🧹 Limpeza e Tratamento de Dados

A preparação dos dados começou com a leitura do arquivo `TelecomX_Data.json` e sua conversão para um DataFrame usando a biblioteca **pandas**. Devido à presença de dados aninhados, foi utilizada a função `pd.json_normalize`, sendo necessário converter o DataFrame para um dicionário com `orient='records'`.

Em seguida, realizou-se uma análise inicial para verificar a existência de valores nulos, duplicatas e inconsistências por meio da inspeção dos valores únicos por coluna. Nessa etapa, identificou-se que a coluna `Churn` possuía registros vazios, que foram removidos com o método `.query()`.

As colunas `account.Charges.Total` e `customer.tenure`, inicialmente no formato texto, foram convertidas para valores numéricos utilizando `pd.to_numeric`, tratando automaticamente valores inválidos como nulos (`errors='coerce'`).

Também foi criada uma nova métrica chamada `Total.Day`, que representa o gasto médio diário do cliente. Essa métrica foi calculada a partir do valor total de cobranças dividido pelo tempo de permanência (em meses), ajustado para um período de 30 dias. Qualquer versão anterior dessa coluna foi removida para evitar duplicidade.

---

### 📊 Análise Exploratória de Dados

Inicialmente, foram extraídas estatísticas descritivas (média, mediana e desvio padrão) para fornecer uma visão geral dos dados, servindo de base para as análises comparativas e visuais.

Para gerar os gráficos, foi utilizada a biblioteca **matplotlib**. Com o objetivo de manter um padrão visual e facilitar a geração dos gráficos, foi criada uma função que segmenta os dados com base na variável `Churn`.

---

#### 📈 Análise dos Gráficos

O tipo de gráfico construido para a análise foi um histogrâma, um gráfico de barras que apresenta a distribuição da frequência dos dados em determinado intervalo, representando a quantidade de dados em cada intervalo por meio da altura da barra.

---

#### Gráfico: Meses de Permanência dos Clientes

![alt text](<../Gráficos/Histograma de Tenure por Churn.png>)

Este gráfico revela que a maioria dos clientes que cancelaram o serviço permaneceu por poucos meses, o que pode indicar uma experiência inicial negativa ou um serviço aquém das expectativas para novos assinantes. Já os clientes antigos, com contratos mais longos, parecem permanecer mais tempo, possivelmente devido à familiaridade com o serviço ou comodidade.

---

#### Contagem de evasão variaveis categóricas

![alt text](<../Gráficos/Histograma de customer.gender por Churn.png>)
![alt text](<../Gráficos/Histograma de account.Contract por Churn.png>)
![alt text](<../Gráficos/Histograma de account.PaymentMethod por Churn.png>)
![alt text](<../Gráficos/Histograma de internet.InternetService por Churn.png>)

Embora esses gráficos não revelem grandes discrepâncias entre quem cancelou ou não o serviço, eles ajudam a identificar padrões de comportamento e concentração de clientes. Isso é útil para entender quais perfis devem ser priorizados em estratégias de retenção.

Por exemplo:
- Em relação ao **gênero**, a evasão é praticamente equilibrada.
- A maioria dos clientes utiliza **contratos mensais**, **internet via fibra óptica** e realiza pagamentos por **cheque eletrônico** — áreas que merecem atenção.

---

#### Contagem de evasão variaveis numéricas

Primeiramente vamos análisar a o quanto as pessoas gastaram pelo serviço ao longo de seu tempo de contrato.

![alt text](<../Gráficos/Histograma de account.Charges.Total por Churn.png>)

Este gráfico confirma que os clientes que cancelaram o serviço geralmente gastaram menos, refletindo a tendência já observada de cancelamentos precoces (menos meses de contrato).

---

![alt text](<../Gráficos/Histograma de Total.Day por Churn.png>)

Este gráfico traz um insight interessante: os clientes que permaneceram na empresa tendem a pagar entre **R$0,50 e R$1,00** por dia, enquanto muitos dos que cancelaram estão na faixa de **R$2,70 a R$3,70** por dia. Isso pode indicar que o custo percebido do serviço em relação ao valor entregue não está sendo bem aceito por novos clientes.

---

### ✅ Conclusões e Insights

- A maioria dos clientes que cancelam o serviço está no perfil “médio” de uso e gasto, ou seja, utilizam o pacote padrão.
- Isso sugere que o serviço básico não está atendendo às expectativas dos novos assinantes, levando à frustração e ao cancelamento precoce.
- Clientes antigos, por outro lado, tendem a permanecer, o que pode indicar fidelidade, resistência à mudança ou satisfação com serviços personalizados.

---

### 💡 Recomendações

- **Melhorar o serviço básico** para atender melhor aos novos assinantes.
- **Investir em campanhas de onboarding** e comunicação clara dos benefícios desde o início do contrato.
- **Aprimorar o atendimento inicial ao cliente**, garantindo uma boa primeira impressão e suporte rápido em caso de problemas.
- **Analisar o custo-benefício percebido pelos clientes**, especialmente na faixa de gasto diário mais elevada.